In [161]:
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing  import LabelEncoder, OneHotEncoder
from datetime import datetime



In [3]:
# --데이터 전처리
# 업체규모(EMP_CNT → 결측값 존재)
# 업력(ESTB_DATE → 결측값 존재)
# 매출액 – 상승추세
# 상장,비상장
# 휴폐업 이력 존재 유무

# 외부데이터
# 산업동향 – 산업위험
# ->통계청 업종별 경기 전망
# 기업 입지 - 도심지역, 외곽지역 (bizno 사업자등록번호 조회)
# (시장위험 – 경기,금리,환율,유가)

# 데이터 가져오기

In [2]:
wb1 = openpyxl.load_workbook('./numble_data/휴폐업 중소법인 재무보유.xlsx')
wb2 = openpyxl.load_workbook('./numble_data/액티브 중소법인 재무보유.xlsx', read_only=True)

In [3]:
sheet_rest = wb1["기업개요"]
sheet_rest_record=wb1['휴폐업이력']

In [8]:
#(휴폐업) 총 6739개
df_rest = pd.DataFrame(sheet_rest.values)
df_rest = df_rest.fillna('NaN')
df_rest.columns = df_rest.iloc[0,:]
df_rest = df_rest.iloc[1:, :]
df_rest = df_rest.reset_index(drop=True)
df_rest.head()

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
0,1268145633,(주),용진테크,NaN,"Yongjin Tech Co., Ltd.",M,2,2,1,1,...,N,,20000529,01,NaN,자동차부품 제조판매업,NaN,www.yongjintech.com,안상동,40
1,1388101749,NaN,신구건설,(주),"SHINKU C0NSTRUCTION CO.,LTD.",M,2,2,1,1,...,N,,19850110,01,NaN,건축자재 판매업,NaN,www.shinku.co.kr,소재필,13
2,1068179240,(주),석정미디어,NaN,sung jung,M,2,2,1,1,...,N,,19990601,01,NaN,"토너, 잉크, 프린터 외 컴퓨터 관련 주변기기",NaN,NaN,이상호,1
3,2248114507,(주),새한실업,NaN,"SAEHAN INDUSTRY CO.,LTD.",M,2,2,1,1,...,N,,19980522,01,NaN,금속 캔 및 기타 포장용기 제조업,NaN,NaN,김정남외 1명,6
4,1168132785,(주),석영엔터프라이즈,NaN,"Seokyoung Enterprise Co.,Ltd.",M,2,2,1,1,...,N,,19930226,01,NaN,스마트트레이 외,NaN,www.systeel.co.kr,박성진,40


In [170]:
#(휴폐업이력)데이터 총 6840개 - 여러번 폐업한 기업 찾아보기
df_rest_record = pd.DataFrame(sheet_rest_record.values)
df_rest_record = df_rest_record.fillna('NaN')
df_rest_record.columns = df_rest_record.iloc[0,:]
df_rest_record = df_rest_record.iloc[1:, :]
df_rest_record.head()

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
1,1268145633,99991231,20220518,3,20220518
2,1388101749,99991231,20210319,3,20210319
3,1068179240,99991231,20200731,3,20191213
4,2248114507,99991231,20210531,3,20210531
5,1168132785,99991231,20210630,3,20210630


In [11]:
sheet_active = wb2['기업개요_외감']
sheet_active_rest = wb2['휴폐업이력']

In [13]:
#(액티브) 외감기업
df_active= pd.DataFrame(sheet_active.values)
df_active = df_active.fillna('NaN')
df_active.columns = df_active.iloc[0, :]
df_active = df_active.iloc[1:, :]
df_active = df_active.reset_index(drop=True)
df_active.head()

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
0,1138111739,NaN,한일가스산업,(주),"HANIL GAS IND. CO.,LTD.",M,2,2,1,1,...,N,,19660419,01,NaN,LP Gas 충전 및 판매업,NaN,www.hanilgas.com,홍순철외 1명,43
1,2078132193,(주),엠피아이,NaN,"EMPIALEE CO.,LTD.",M,2,2,1,1,...,N,,19910806,01,NaN,가죽 및 대용가죽제품 제조업,NaN,www.mastercnd.co.kr,이스테판상수외 1명,35
2,1318167565,NaN,도영운수,(주),"DOYOUNG TRANSPORTATION CO.,LTD.",M,2,2,1,1,...,N,,20011030,01,NaN,시내버스업,NaN,transport@naver.com,김명화,149
3,5048128251,(주),중원산업,NaN,"JOONGWON INDUSTRIAL CO.,LTD.",M,2,2,1,1,...,N,,19990112,01,NaN,자동차 부품 제조,NaN,www.koreajw.com,김명구,75
4,1098164260,(주),카타나골프,NaN,"CATANA GOLF CO.,LTD.",M,2,2,1,1,...,N,,20000531,01,NaN,골프용품 전반의 제조 및 수출업,NaN,NaN,김홍득,3


In [169]:
#(액티브) 휴폐업이력 - 4828개
df_active_rest = pd.DataFrame(sheet_active_rest.values)
df_active_rest = df_active_rest.fillna('NaN')
df_active_rest.columns = df_active_rest.iloc[0, :]
df_active_rest = df_active_rest.iloc[1:, :]
df_active_rest = df_active_rest.reset_index(drop=True)
df_active_rest.head()

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,5088115592,99991231,20150209,5,NaN
1,5088115592,20150208,20200731,2,NaN
2,1138127100,99991231,20121105,5,NaN
3,1138127100,20121104,20200731,2,NaN
4,1138127100,20121004,20200731,5,NaN


In [12]:
# 액티브
# -액티브 외감 df_active_big 28982 
# -휴폐업이력 4828개
# -(액티브 df_active 198651) - 참고자료

# 휴폐업
# -기업개요 - 휴폐업 데이터 df_rest 6739
# -휴폐업이력  df_rest_record 6840 (여러번 휴폐업한 기업 찾기)

# EDA

In [14]:
#예측을 한다, 액티브+휴폐업 기업 섞여있을 때 폐업인지 아닌지 구분을 하는 것.
#일단 예측을 해보고 외부데이터를 추가하면서 성능 높아지는지 체크해야할듯

#액티브 데이터 n개, 휴폐업 데이터 n개 -> x값
#폐업:1, 정상영업:0 으로 정답y데이터 만들기
#문자열 데이터 인코딩하기


In [15]:
#휴폐업데이터에서 산업코드1로 groupby하기 - 그룹 총 1037개(너무많음)
#업종(M,O,V)별로 예측따로하기?
df1 = df_rest.groupby('BZ_TYP').count()
df1

,BIZ_NO,CMP_PFIX_NM,CMP_NM,CMP_SFIX_NM,CMP_ENM,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
BZ_TYP,,,,,,,,,,,,,,,,,,,,,
M,6698,4830,6698,1261,6654,6698,6698,6698,6698,6698,...,6698,6698,6133,6698,49,6530,143,2107,6697,6698
O,38,15,38,22,38,38,38,38,38,38,...,38,38,38,38,0,38,1,17,38,38
V,3,0,3,3,3,3,3,3,3,3,...,3,3,3,3,0,3,0,1,3,3


In [18]:
df3 = df_active.groupby('IND_CD_ORDR').count()
df3

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT
IND_CD_ORDR,,,,,,,,,,,,,,,,,,,,,
10,28972,18578,28972,9165,28955,28972,28972,28972,28972,28972,...,28972,28972,27898,28972,159,28809,1645,17764,28972,28972
,10,2,10,4,10,10,10,10,10,10,...,10,10,10,10,0,10,0,1,9,10


In [21]:
df_test=df_active.loc[df_active['IND_CD_ORDR']=='  ']
df_test

,BIZ_NO,CMP_PFIX_NM,CMP_NM1,CMP_SFIX_NM,CMP_ENM,BZ_TYP,CMP_SCL,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,...,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT


In [108]:
df_active.isnull().sum()
#IND_CD_ORDR에 null값은 없는데 위에 그룹화한거에 왜 공백...? 저게 널값인가?


0
BIZ_NO             0
CMP_PFIX_NM        0
CMP_NM1            0
CMP_SFIX_NM        0
CMP_ENM            0
BZ_TYP             0
CMP_SCL            0
PBCO_GB            0
PSN_CORP_GB        0
HDOF_BR_GB         0
FR_IVST_CORP_YN    0
VENT_YN            0
LIST_CD            0
IND_CD_ORDR        0
IND_CD1            0
IND_CD2            0
IND_CD3            0
PB_ORG_TYP         0
MDSCO_PRTC_YN      0
HDOF_CMP_CD        0
ESTB_DATE          0
ESTB_GB            0
LIST_DATE          0
MN_BIZ_CONT        0
NATN_NM            0
HOMEPAGE_URL       0
CEO_NM             0
EMP_CNT            0
dtype: int64

In [110]:
#결측치 확인
#휴폐업 이력이 있는 회사 df_rest_record 총 6739개
#휴폐업 데이터 df_rest 6840
#액티브 외감 df_active_big 28982 
#((액티브 df_active 198651))

df_active.isnull().sum().sum()


0

In [ ]:
# 사업자번호	결산년월	유동자산	매출채권	비유동자산	유형자산	자산총계	유동부채
# 비유동부채	부  채  총  계	자본금	이익잉여금(결손금）	자본총계	매출액	판매비와관리비
# 영업이익（손실）	법인세비용차감전순손익	법인세비용	당기순이익(손실)	기업순이익률(%)
# 유보액/총자산(%)	유보액/납입자본(%)	매출액총이익률(%)	매출액영업이익률(%)	매출액순이익률(%)
# 수지비율(%)	경상수지비율	영업비율(%)	금융비용대매출액비율(%	금융비용대부채비율(%)
# 금융비용대총비용비율(%	부채비율(%)	차입금의존도(%)	자기자본비율(%)	순운전자본비율(%)	
# 유동부채비율(%)	비유동부채비율(%)	부채총계대 매출액(%)	총자본회전율(회)	재고자산회전율(회)
# 매출채권회전율(회)	매입채무회전율(회)	미수금	매출원가	무형자산	재고자산

## 전처리 시작

In [111]:
#사용할 컬럼들
select_columns = ['BIZ_NO', 'BZ_TYP','NATN_NM', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN', 'LIST_CD', 'IND_CD_ORDR', 'IND_CD1', 'IND_CD2', 'IND_CD3', 'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE', 'ESTB_GB', 'LIST_DATE', 'EMP_CNT']

In [112]:
len(select_columns)

19

In [141]:
df_rest_select = df_rest.loc[:, select_columns]
df_active_select = df_active.loc[:, select_columns]
df_active_select

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT
0,1138111739,M,NaN,2,1,1,N,N,NaN,10,47712,NaN,NaN,N,,19660419,01,NaN,43
1,2078132193,M,NaN,2,1,1,N,N,NaN,10,15190,NaN,NaN,N,,19910806,01,NaN,35
2,1318167565,M,NaN,2,1,1,N,N,NaN,10,49212,NaN,NaN,N,,20011030,01,NaN,149
3,5048128251,M,NaN,2,1,1,N,N,NaN,10,30391,NaN,NaN,N,,19990112,01,NaN,75
4,1098164260,M,NaN,2,1,1,N,N,NaN,10,46800,NaN,NaN,N,,20000531,01,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,M,NaN,2,1,1,N,N,NaN,,NaN,NaN,NaN,N,,20191206,01,NaN,0
28978,8028701430,M,NaN,2,1,1,N,N,NaN,10,68122,NaN,NaN,N,,20200402,01,NaN,1
28979,4268501421,M,NaN,2,1,2,N,N,NaN,10,72100,NaN,NaN,N,1220726,19900108,01,NaN,81
28980,6518502462,M,NaN,2,1,2,N,N,NaN,10,23322,NaN,NaN,N,1583116,20220615,01,NaN,26


In [114]:
#라벨인코딩
#  ['FR_IVST_CORP_YN' - N이 0
#   'VENT_YN' - N이 0
#   'MDSCO_PRTC_YN'] N이 0
# 'NATN_NM' 국가 총 64개

In [142]:
encode_features = ['FR_IVST_CORP_YN' , 'VENT_YN', 'MDSCO_PRTC_YN', 'NATN_NM']

for df in [df_rest_select, df_active_select]:
    for feature in encode_features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])

df_active_select

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT
0,1138111739,M,0,2,1,1,0,0,NaN,10,47712,NaN,NaN,0,,19660419,01,NaN,43
1,2078132193,M,0,2,1,1,0,0,NaN,10,15190,NaN,NaN,0,,19910806,01,NaN,35
2,1318167565,M,0,2,1,1,0,0,NaN,10,49212,NaN,NaN,0,,20011030,01,NaN,149
3,5048128251,M,0,2,1,1,0,0,NaN,10,30391,NaN,NaN,0,,19990112,01,NaN,75
4,1098164260,M,0,2,1,1,0,0,NaN,10,46800,NaN,NaN,0,,20000531,01,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,M,0,2,1,1,0,0,NaN,,NaN,NaN,NaN,0,,20191206,01,NaN,0
28978,8028701430,M,0,2,1,1,0,0,NaN,10,68122,NaN,NaN,0,,20200402,01,NaN,1
28979,4268501421,M,0,2,1,2,0,0,NaN,10,72100,NaN,NaN,0,1220726,19900108,01,NaN,81
28980,6518502462,M,0,2,1,2,0,0,NaN,10,23322,NaN,NaN,0,1583116,20220615,01,NaN,26


In [137]:
#국가(NATN_NM) 갯수
countryDF = pd.DataFrame(df_active_select['NATN_NM'].value_counts())
country_list=countryDF.index.tolist()
len(country_list)
#국가를 대륙별로 모을까.?? -> 일단 라벨인코딩 완료

64

In [143]:
#원핫인코딩
#PBCO_GB	공기업구분 1.공기업 2.일반기업
# PSN_CORP_GB	개인법인구분 1.법인 2.개인
# HDOF_BR_GB	본점지점구분 1.본점 2.지점

#하나의 컬럼마다 2차원데이터프레임으로 존재해야해서 원핫인코딩 불가!
#라벨인코딩도 굳이 안해도 될 듯해서 일단 pass


In [144]:
#LIST_CD 상장코드 - 상장유무로 0,1로 바꾸기

df_active_select['LIST_CD'] = df_active_select['LIST_CD'].replace('NaN',0)

# for df in [df_rest_select, df_active_select]:
#     if df['LIST_CD']=='NaN':
#         df['LIST_CD']=0
#     else:
#         df['LIST_CD']=1
df_active_select

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT
0,1138111739,M,0,2,1,1,0,0,0,10,47712,NaN,NaN,0,,19660419,01,NaN,43
1,2078132193,M,0,2,1,1,0,0,0,10,15190,NaN,NaN,0,,19910806,01,NaN,35
2,1318167565,M,0,2,1,1,0,0,0,10,49212,NaN,NaN,0,,20011030,01,NaN,149
3,5048128251,M,0,2,1,1,0,0,0,10,30391,NaN,NaN,0,,19990112,01,NaN,75
4,1098164260,M,0,2,1,1,0,0,0,10,46800,NaN,NaN,0,,20000531,01,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,M,0,2,1,1,0,0,0,,NaN,NaN,NaN,0,,20191206,01,NaN,0
28978,8028701430,M,0,2,1,1,0,0,0,10,68122,NaN,NaN,0,,20200402,01,NaN,1
28979,4268501421,M,0,2,1,2,0,0,0,10,72100,NaN,NaN,0,1220726,19900108,01,NaN,81
28980,6518502462,M,0,2,1,2,0,0,0,10,23322,NaN,NaN,0,1583116,20220615,01,NaN,26


In [145]:
#상장기업 89개
LISTCD = df_active_select.loc[df_active_select['LIST_CD']!=0]
len(LISTCD)

89

In [146]:
#상장기업인지 아닌지 컬럼 하나 추가
df_active_select['Listing'] = df_active_select['LIST_CD']

#조건식을 따로 작성!!!
listing_condition = (df_active_select['Listing'] != 0)
# df_active_select['Listing'] != 0
#Listing컬럼이 0이 아니면(상장기업), 1을 넣는 코드 

df_active_select.loc[listing_condition, 'Listing']=1
df_active_select



,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT,Listing
0,1138111739,M,0,2,1,1,0,0,0,10,47712,NaN,NaN,0,,19660419,01,NaN,43,0
1,2078132193,M,0,2,1,1,0,0,0,10,15190,NaN,NaN,0,,19910806,01,NaN,35,0
2,1318167565,M,0,2,1,1,0,0,0,10,49212,NaN,NaN,0,,20011030,01,NaN,149,0
3,5048128251,M,0,2,1,1,0,0,0,10,30391,NaN,NaN,0,,19990112,01,NaN,75,0
4,1098164260,M,0,2,1,1,0,0,0,10,46800,NaN,NaN,0,,20000531,01,NaN,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28977,8048101453,M,0,2,1,1,0,0,0,,NaN,NaN,NaN,0,,20191206,01,NaN,0,0
28978,8028701430,M,0,2,1,1,0,0,0,10,68122,NaN,NaN,0,,20200402,01,NaN,1,0
28979,4268501421,M,0,2,1,2,0,0,0,10,72100,NaN,NaN,0,1220726,19900108,01,NaN,81,0
28980,6518502462,M,0,2,1,2,0,0,0,10,23322,NaN,NaN,0,1583116,20220615,01,NaN,26,0


In [147]:
df_active_select.groupby('Listing').count()

,BIZ_NO,BZ_TYP,NATN_NM,PBCO_GB,PSN_CORP_GB,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD_ORDR,IND_CD1,IND_CD2,IND_CD3,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,EMP_CNT
Listing,,,,,,,,,,,,,,,,,,,
0,28893,28893,28893,28893,28893,28893,28893,28893,28893,28893,28893,28893,28893,28893,28893,28893,28893,28893,28893
1,89,89,89,89,89,89,89,89,89,89,89,89,89,89,89,89,89,89,89


In [149]:
df_active_select.dtypes

0
BIZ_NO             object
BZ_TYP             object
NATN_NM             int32
PBCO_GB            object
PSN_CORP_GB        object
HDOF_BR_GB         object
FR_IVST_CORP_YN     int32
VENT_YN             int32
LIST_CD            object
IND_CD_ORDR        object
IND_CD1            object
IND_CD2            object
IND_CD3            object
MDSCO_PRTC_YN       int32
HDOF_CMP_CD        object
ESTB_DATE          object
ESTB_GB            object
LIST_DATE          object
EMP_CNT            object
Listing            object
dtype: object

In [154]:
#type을 float로 바꿀 컬럼들
float_columns = ['NATN_NM', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN', 'IND_CD_ORDR', 'MDSCO_PRTC_YN', 'ESTB_GB', 'EMP_CNT']

In [155]:
#type 변경
for columns in float_columns:
    df_active_select[columns] = pd.to_numeric(df_active_select[columns])
    df_rest_select[columns] = pd.to_numeric(df_rest_select[columns])

In [157]:
df_rest_select.dtypes

0
BIZ_NO              object
BZ_TYP              object
NATN_NM              int32
PBCO_GB              int64
PSN_CORP_GB          int64
HDOF_BR_GB           int64
FR_IVST_CORP_YN      int32
VENT_YN              int32
LIST_CD             object
IND_CD_ORDR        float64
IND_CD1             object
IND_CD2             object
IND_CD3             object
MDSCO_PRTC_YN        int32
HDOF_CMP_CD         object
ESTB_DATE           object
ESTB_GB              int64
LIST_DATE           object
EMP_CNT            float64
dtype: object

In [158]:
#직원수의 전체 데이터 요약 (중간값, 평균값, min, max) 
df_active_select['EMP_CNT'].describe()

count    28248.000000
mean        62.770886
std        137.411139
min          0.000000
25%          7.000000
50%         30.000000
75%         67.000000
max       5099.000000
Name: EMP_CNT, dtype: float64

In [159]:
df_rest_select['EMP_CNT'].describe()

count     6051.000000
mean        25.987275
std        222.573079
min          0.000000
25%          3.000000
50%          7.000000
75%         17.000000
max      16310.000000
Name: EMP_CNT, dtype: float64

In [ ]:
#구간화 - 직원수EMP_CNT	, 설립일자ESTB_DATE
#그룹화? - BZ_TYP, IND_CD1	
#빼고 완료

In [190]:
df_active_rest

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,5088115592,99991231,20150209,5,NaN
1,5088115592,20150208,20200731,2,NaN
2,1138127100,99991231,20121105,5,NaN
3,1138127100,20121104,20200731,2,NaN
4,1138127100,20121004,20200731,5,NaN
...,...,...,...,...,...
4822,6628701486,20220913,20220830,2,NaN
4823,6098801857,99991231,20220921,6,NaN
4824,6098801857,20220920,20220607,3,20220607
4825,4988102183,99991231,20220705,5,NaN


In [203]:
df_active_rest['CLSBZ_GB'] = pd.to_numeric(df_active_rest['CLSBZ_GB'])
df_active_rest

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,5088115592,99991231,20150209,5,NaN
1,5088115592,20150208,20200731,2,NaN
2,1138127100,99991231,20121105,5,NaN
3,1138127100,20121104,20200731,2,NaN
4,1138127100,20121004,20200731,5,NaN
...,...,...,...,...,...
4822,6628701486,20220913,20220830,2,NaN
4823,6098801857,99991231,20220921,6,NaN
4824,6098801857,20220920,20220607,3,20220607
4825,4988102183,99991231,20220705,5,NaN


In [205]:
#휴폐업이력sheet에 있는 사업자등록번호랑 연결해서
#단순하게 2,3번을 count해서 휴폐업 횟수 구해서 close_count컬럼만들기

#사업자등록번호 겹치는거 다 하나씩만 남기고, 새 컬럼에 2,3번 count하나 만들어서 값 넣고
#join(outer)로 BIZ_NO로 연결해서 df만들기

close_condition = ('CLSBZ_GB'==2) 
df_active_rest=df_active_rest.set_index(['BIZ_NO','CLSBZ_GB'])
# df_active_rest.loc[close_condition, 'close_YN']=True
df_active_rest['CLSBZ_GB'].


#내가 하고 싶은 건, 
#사업자등록번호별로 CLSBZ_GB에 2나 3이 몇번 있는지 구해서 컬럼 새로 만들어서 추가
#그리고 휴폐업 유무 컬럼도 추가해서 0과 1 값 주고싶음.

0                    END_DATE STRT_DATE STAT_OCR_DATE
BIZ_NO     CLSBZ_GB                                  
5088115592 5         99991231  20150209           NaN
           2         20150208  20200731           NaN
1138127100 5         99991231  20121105           NaN
           2         20121104  20200731           NaN
           5         20121004  20200731           NaN
...                       ...       ...           ...
6628701486 2         20220913  20220830           NaN
6098801857 6         99991231  20220921           NaN
           3         20220920  20220607      20220607
4988102183 5         99991231  20220705           NaN
           2         20220704  20220127           NaN

[4827 rows x 3 columns]

In [175]:
df_active_rest

,BIZ_NO,END_DATE,STRT_DATE,CLSBZ_GB,STAT_OCR_DATE
0,5088115592,99991231,20150209,5,NaN
1,5088115592,20150208,20200731,2,NaN
2,1138127100,99991231,20121105,5,NaN
3,1138127100,20121104,20200731,2,NaN
4,1138127100,20121004,20200731,5,NaN
...,...,...,...,...,...
4822,6628701486,20220913,20220830,2,NaN
4823,6098801857,99991231,20220921,6,NaN
4824,6098801857,20220920,20220607,3,20220607
4825,4988102183,99991231,20220705,5,NaN


In [ ]:
#휴업기간 구해서 컬럼 추가

#휴폐업이력에서 발생일자가 존재하지 않는 기업이 너무 많음 -> 휴업기간을 구할 수 없음
#-> 



In [ ]:
#업력 컬럼 만들기
#업력=설립날짜-오늘날짜-휴업기간

now=datetime.now()
date_establish = datetime.strptime(컬럼명,"%Y%m%d")
업력기간=(now-date_establish-휴업기간).days
업력=업력기간//365 


In [ ]:
#휴폐업 정답컬럼추가 - 0/1
#휴폐업 횟수 컬럼 추가
df_active_select['close']=
df_active_select['close_count']=

#정답(y)컬럼을 만들때, 한번이라도 휴폐업한 기업이면 1값을 주는게 맞나? 
#아니면 현재기준으로 해야하나..

## 시각화

In [160]:
#type을 float로 바꾼 컬럼들
float_columns = ['NATN_NM', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB', 'FR_IVST_CORP_YN', 'VENT_YN', 'IND_CD_ORDR', 'MDSCO_PRTC_YN', 'ESTB_GB', 'EMP_CNT']

Index(['BIZ_NO', 'BZ_TYP', 'NATN_NM', 'PBCO_GB', 'PSN_CORP_GB', 'HDOF_BR_GB',
       'FR_IVST_CORP_YN', 'VENT_YN', 'LIST_CD', 'IND_CD_ORDR', 'IND_CD1',
       'IND_CD2', 'IND_CD3', 'MDSCO_PRTC_YN', 'HDOF_CMP_CD', 'ESTB_DATE',
       'ESTB_GB', 'LIST_DATE', 'EMP_CNT', 'Listing'],
      dtype='object', name=0)